In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder()
  .appName("Jupyter")
  .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://afd93b5c3cd4:4041
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1681114238034)
SparkSession available as 'spark'


23/04/10 08:10:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@ffaab28


In [28]:
import org.apache.spark.sql

val receiptSchema = new sql.types.StructType()
    .add("transaction_hash",sql.types.StringType,true)
    .add("transaction_index",sql.types.LongType,true)
    .add("block_hash",sql.types.StringType,true)
    .add("block_number",sql.types.StringType,true)
    .add("cumulative_gas_used",sql.types.LongType,true)
    .add("gas_used",sql.types.LongType,true)
    .add("contract_address",sql.types.StringType,true)
    .add("root",sql.types.StringType,true)
    .add("status",sql.types.LongType,true)
    .add("effective_gas_price",sql.types.LongType,true)

val logSchema = new sql.types.StructType()
    .add("log_index",sql.types.LongType,true)
    .add("transaction_hash",sql.types.StringType,true)
    .add("transaction_index",sql.types.LongType,true)
    .add("block_hash",sql.types.StringType,true)
    .add("block_number",sql.types.LongType,true)
    .add("address",sql.types.StringType,true)
    .add("data",sql.types.StringType,true)
    .add("topics",sql.types.StringType,true)

val startBlock = "00000000"
val endBlock = "00005999"

val receiptDf = spark.read.format("csv")
      .option("header", "true")
      .schema(receiptSchema)
      .load("/home/iceberg/warehouse/receipts/start_block=" + startBlock + 
            "/end_block=" + endBlock + 
            "/receipts_" + startBlock + "_" + endBlock+ ".csv")

val logDf = spark.read.format("csv")
      .option("header", "true")
      .schema(logSchema)
      .load("/home/iceberg/warehouse/logs/start_block=" + startBlock + 
            "/end_block=" + endBlock + 
            "/logs_" + startBlock + "_" + endBlock+ ".csv")

val mergeDf = receiptDf.select("contract_address").distinct().union(
    logDf.select(col("address").alias("contract_address")).distinct()
)
mergeDf.filter(mergeDf("contract_address").isNotNull).show()


mergeDf.write.option("header", true)
 .csv("/home/iceberg/warehouse/contracts/start_block=" + startBlock + 
            "_end_block=" + endBlock)

+--------------------+
|    contract_address|
+--------------------+
|0x000000000000000...|
|0x000000000000000...|
|0x000000000000000...|
|0x000000000000000...|
|0x000000000000000...|
|0x000000000000000...|
|0x000000000000000...|
+--------------------+



import org.apache.spark.sql
receiptSchema: org.apache.spark.sql.types.StructType = StructType(StructField(transaction_hash,StringType,true),StructField(transaction_index,LongType,true),StructField(block_hash,StringType,true),StructField(block_number,StringType,true),StructField(cumulative_gas_used,LongType,true),StructField(gas_used,LongType,true),StructField(contract_address,StringType,true),StructField(root,StringType,true),StructField(status,LongType,true),StructField(effective_gas_price,LongType,true))
logSchema: org.apache.spark.sql.types.StructType = StructType(StructField(log_index,LongType,true),StructField(transaction_hash,StringType,true),StructField(transaction_index,LongType,true),StructField(block_hash,StringType,true),StructField(block_number,LongType,true),StructField(add...
